In [55]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [57]:
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    row_accumulator = []

    def splitListToRows(row, separator):
        split_row = str(row[target_column]).split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1, args = (separator, ))
    new_df = pd.DataFrame(row_accumulator)
    return new_df

In [68]:
#Match missing works to their composers before concatenating.
import pandas as pd

cursor = conn.cursor()  
query = "select name, work_type, uri, " + \
        "case when aka2 IS NULL then aka " + \
        "else aka || '; ' || aka2 end as aka " + \
        "from " + \
        "  (select name, aka, work_type, uri, string_agg(work,'; ') as aka2 " + \
        "  from " + \
        "    (SELECT DISTINCT w.name, w.aka, w.work_type, w.uri, lm.work " + \
        "     FROM works w " + \
        "       LEFT JOIN listing_mappers lm ON w.uri = lm.work_uri " + \
        "     WHERE w.uri != 'W1' " + \
        "     ORDER BY w.name " + \
        "    ) w " + \
        "  group by name, aka, work_type, uri) w"
cursor.execute(query)

w_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'work_type', 'uri', 'aka'])
w_df = splitDataFrameList(w_df, 'aka', '; ')
w_df = w_df.drop_duplicates()

cursor = conn.cursor()  
query = "SELECT lower(l.composer) as composer, lower(l.work) as work, l.map_key, " + \
        "SUM(CASE WHEN l.source_id IN (6, 11) THEN CAST(l.additional_text AS INT) ELSE 1 END) as listings " + \
        "FROM listings l " + \
        "LEFT JOIN listing_mappers lm ON l.map_key = lm.map_key " + \
        "WHERE lm.work_uri IS NULL and l.source_id != 6 " + \
        "GROUP BY l.composer, l.work, l.map_key "
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['composer', 'work', 'map_key', 'listings'])

In [69]:
# l_df[l_df['work']=='sweeney todd']
# l_df.head(10)
# l_df.shape
# l_df = l_df[l_df['work']=='mazeppa']
l_df.shape

(5138, 4)

In [70]:
#Update missing keys for listings
import Levenshtein
map_dict = {}
for idx,row in l_df.iterrows():
    
    if idx%250==0:
        print(idx)
        
    comp_key = row['composer']+'---'+row['work']
    if map_dict.get(comp_key):
        None
    else:
        map_dict[comp_key] = {}

        for idx2,w in w_df.iterrows():        
            dist_to_work = Levenshtein.distance(row['work'].lower(), w['aka'].lower())
            try:                
                distance = map_dict[comp_key].get(w['uri'])
                if dist_to_work < distance:
                    map_dict[comp_key][w['uri']] = dist_to_work
            except:
                map_dict[comp_key][w['uri']] = dist_to_work

0


KeyboardInterrupt: 

In [67]:
import pandas as pd

closest_works={}

for idx,key in enumerate(list(map_dict.keys())):
            
    w_dict = map_dict.get(key)
    key_min = min(w_dict.keys(), key=(lambda k: w_dict[k]))
    dist = w_dict.get(key_min)
    norm_dist = w_dict.get(key_min)/max(len(key), 1)    
    closest_works[key] = [key_min, dist, norm_dist]
    
cursor = conn.cursor()  
query = "SELECT w.name as name, w.uri, w.aka, w.work_type FROM works w"        
cursor.execute(query)
works_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'uri', 'aka', 'work_type'])


df = pd.DataFrame.from_dict(closest_works, orient='index', columns=['pm_work_uri', 'pm_distance', 'pm_norm_distance'])
df['pm_work_composer'] = df.index
# print(df.shape)
df = df.merge(works_df, left_on=['pm_work_uri'], right_on=['uri'], how='inner')
# print(df.shape)
df = df[['pm_work_uri', 'pm_distance', 'pm_norm_distance', 'pm_work_composer', 'name','uri']]

# df = df[(df['pm_distance']>1) & (df['pm_norm_distance']<0.075)]
df = df[df['pm_distance']==2]
# df = df[df['pm_norm_distance']<0.15]
display(HTML(df.to_html()))
    

,pm_work_uri,pm_distance,pm_norm_distance,pm_work_composer,name,uri
29,Q593319,2,0.222222,---fauvel,L'heure espagnole,Q593319
65,Q8011655,2,0.222222,---elaine,Alcine,Q8011655
89,Q4747995,2,0.285714,---tod!,Amour,Q4747995
102,Q4838185,2,0.285714,---baal,Baby,Q4838185
110,Q4697456,2,0.222222,---caino,Aino,Q4697456
111,Q4697456,2,0.285714,---ping,Aino,Q4697456
138,Q4902482,2,0.250000,---blanc,Bianca,Q4902482
178,Q16833639,2,0.200000,---caritas,Ariane,Q16833639
180,Q16833639,2,0.250000,---trial,Ariane,Q16833639
220,Q5142336,2,0.222222,---floods,Cold Sassy Tree,Q5142336


In [65]:
for idx,row in df.iterrows():
    composer = row['pm_work_composer'].split('---')[0]    
    work = row['pm_work_composer'].split('---')[1]
    map_key = composer+work
    insert = 'INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('
    values = "'" + work.replace("'", "''") + "', '" +  composer.replace("'", "''") + "', '" + map_key.replace("'", "''") + "', '" + row['pm_work_uri'] + "'); "
    commit = 'COMMIT;'    
    print(insert + values + commit)

INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('kitezh', '', 'kitezh', 'Q1038931'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('nosferatu', '', 'nosferatu', 'Q7061798'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('otto', '', 'otto', 'Q690606'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('pirates', '', 'pirates', 'Q1215463'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('susannens geheimnis', '', 'susannens geheimnis', 'Q521233'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('straszny dwór', '', 'straszny dwór', 'Q468161'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('sunken garden', '', 'sunken garden', 'Q3521497'); COMMIT;
INSERT INTO LISTING_MAPPERS (WORK, COMPOSER, MAP_KEY, WORK_URI) VALUES ('the demon', '', 'the demon', 'Q2623107'); COMMIT;
INS

In [21]:
i=0
for key in map_dict:
    i += len(list(map_dict.get(key).keys()))
# l_df.shape
i

201657372

In [19]:
#Import the worked dict
import pickle
filename = 'levenstein_dict'

map_dict = pickle.load(open(filename, "rb" ))